## TRADE DATA

In [2]:
import pandas as pd
import datetime

def process_csv(filepath, product_index):
    """
    This function takes the file path and product index as inputs and returns the filtered trade data dataframe
    for a particular product.
    
    Parameters:
    filepath (str) : file path for the trade data file
    product_index (int) : index of the product in the list of all products
    
    Returns:
    df_product_filtered (pandas dataframe) : dataframe for the filtered product
    """
    
    # Read the CSV file and convert the "TIME_PERIOD" column to datetime
    trade_data = pd.read_csv(filepath)
    trade_data["TIME_PERIOD"] = pd.to_datetime(trade_data["TIME_PERIOD"], format = '%Y-%m').dt.date
    
    # Rename columns and remove duplicates
    trade_data = trade_data.rename(columns={"partner": "PARTNER_codes", "declarant": "DECLARANT_codes"})
    trade_data = trade_data.drop_duplicates()
    
    # Get the list of all products
    products = trade_data["product"].drop_duplicates().to_list()
    
    # Filter the data for a particular product
    product_filter = products[product_index]
    
    # Read the partner countries data
    partner_countries = pd.read_csv("trade_data/partners.csv", sep = ";")
    
    # Merge the trade data and partner country data on the "PARTNER_codes" column
    trade_data = pd.merge(trade_data, partner_countries, on = ['PARTNER_codes'], how = 'inner')
    
    # Pivot the trade data to create a multi-index dataframe
    trade_data_pivot = trade_data.pivot(index=['PARTNER_Labels', 'TIME_PERIOD', 'product'], 
                                  columns='indicators', 
                                  values=['OBS_VALUE'])
    
    # Flatten the column names
    trade_data_pivot.columns = trade_data_pivot.columns.map('_'.join)
    
    # Reset the index and remove missing values
    trade_data_pivot = trade_data_pivot.reset_index()
    trade_data_pivot = trade_data_pivot.dropna()
    
    # Filter the data for the selected product
    df = trade_data_pivot
    df_product_filtered = df[df['product'] == product_filter]

    
    return df_product_filtered

#Declare DataFrames 
df_maize_corn_0 = process_csv("trade_data/maize_corn.csv", 0)
df_maize_corn_0['price'] = df_maize_corn_0['OBS_VALUE_VALUE_1000EURO'] / df_maize_corn_0['OBS_VALUE_QUANTITY_TON']

# Keep '10059000' the same but name it 'maize'
df_maize = df_maize_corn_0
df_maize['product'] = 'maize'


In [3]:
df_maize

,PARTNER_Labels,TIME_PERIOD,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price
0,Afghanistan,2021-03-01,maize,0.0,0.04,inf
1,Antigua and Barbuda,2020-01-01,maize,54.0,21.20,0.392593
2,Antigua and Barbuda,2020-04-01,maize,27.0,12.14,0.449630
3,Argentina,2005-01-01,maize,8100.0,937.50,0.115741
4,Argentina,2005-02-01,maize,3172.0,392.55,0.123755
...,...,...,...,...,...,...
5534,Zimbabwe,2007-01-01,maize,0.1,0.13,1.300000
5535,Zimbabwe,2010-12-01,maize,0.3,1.11,3.700000
5536,Zimbabwe,2011-01-01,maize,0.1,0.38,3.800000
5537,Zimbabwe,2011-03-01,maize,0.2,0.38,1.900000


## WEATHER DATA

In [4]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the weather file
df_global_temperature = pd.read_csv('weather_data/global_temperature_data.csv')

# Rename the 'YearMonth' column to 'TIME_PERIOD'
df_global_temperature.rename(columns={'yearmonth': 'TIME_PERIOD', 'partner_labels': 'PARTNER_Labels'}, inplace=True)

# Convert the TIME_PERIOD column to datetime format
df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'], format='%Y%m')

# Convert the TIME_PERIOD column to the desired format
df_global_temperature['TIME_PERIOD'] = df_global_temperature['TIME_PERIOD'].dt.strftime('%Y-%m-%d')

# Drop columns
df_global_temperature.drop(columns=['partner_code', 'weather_countrycode', 'weather_countryname', 'iso_countrycode'], inplace=True)

df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'])
df_global_temperature

,PARTNER_Labels,TIME_PERIOD,AVG_TAVG,MIN_TMIN,MAX_TMAX
0,United Arab Emirates,2005-01-01,186.096154,110.000000,280.000000
1,United Arab Emirates,2005-02-01,204.750000,106.000000,306.500000
2,United Arab Emirates,2005-03-01,238.226667,124.500000,363.666667
3,United Arab Emirates,2005-04-01,280.984568,155.000000,423.333333
4,United Arab Emirates,2005-05-01,321.296296,192.000000,435.500000
...,...,...,...,...,...
21085,Viet Nam,2022-08-01,287.060676,244.363636,348.666667
21086,Viet Nam,2022-09-01,279.751352,233.600000,350.000000
21087,Viet Nam,2022-10-01,246.000000,238.000000,260.000000
21088,Viet Nam,2022-11-01,272.000000,252.000000,304.000000


## OIL DATA

In [5]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the 'oil_price_index_2015_oecd.xlsx' file
df_oil_index = pd.read_excel('oil_prices/oil_price_index_2015_oecd.xlsx', skiprows=[0,1,2,3,4, 9012, 9013, 9014, 9015])
# Forward fill the 'Country' column
df_oil_index['Country'].fillna(method='ffill', inplace=True)
# Rename columns
df_oil_index.rename(columns={'Unnamed: 3': 'CPI_ENERGY'}, inplace=True)
df_oil_index.rename(columns={'Unnamed: 4': 'CPI_ENERGY_HARMONISED'}, inplace=True)
# Delete the 'Unnamed: 2' column
df_oil_index.drop(columns=['Unnamed: 2'], inplace=True)
# Convert the 'Time' column to datetime format
df_oil_index['Time'] = pd.to_datetime(df_oil_index['Time'], format='%b-%Y')
df_oil_index.rename(columns={'Time': 'TIME_INDEX'}, inplace=True)

#Clean import CMO-Historical-Data-Monthly.xlsx

df_commodities_price = pd.read_excel('oil_prices/CMO-Historical-Data-Monthly.xlsx', sheet_name = 'Monthly Prices', skiprows=[0,1,2,3,4,5])
# Rename columns
df_commodities_price.rename(columns={'Unnamed: 0': 'TIME_PERIOD'}, inplace=True)
df_commodities_price.rename(columns={'CRUDE_PETRO': 'OIL_PRICE $/bbl'}, inplace=True)
df_commodities_price['TIME_PERIOD'] = pd.to_datetime(df_commodities_price['TIME_PERIOD'], format='%YM%m')

df_oil_price_full = pd.DataFrame(df_commodities_price, columns=['TIME_PERIOD', 'OIL_PRICE $/bbl'])

df_oil_price = df_oil_price_full.loc[(df_oil_price_full['TIME_PERIOD'] >= '2005-01-01') & (df_oil_price_full['TIME_PERIOD'] <= '2022-10-01')]
df_oil_price

,TIME_PERIOD,OIL_PRICE $/bbl
540,2005-01-01,42.972278
541,2005-02-01,44.818211
542,2005-03-01,50.942879
543,2005-04-01,50.640476
544,2005-05-01,47.826573
...,...,...
749,2022-06-01,116.800000
750,2022-07-01,105.083333
751,2022-08-01,95.973333
752,2022-09-01,88.220000


## FUTURES PRICES (CORN)

In [6]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the weather file
df_futures_corn = pd.read_csv('futures_data/US Corn Futures Historical Data.csv')

# Rename the 'date' column to 'TIME_PERIOD'
df_futures_corn.rename(columns={'Date': 'TIME_PERIOD', 'Price': 'Corn Price Futures'}, inplace=True)

df_futures_corn['TIME_PERIOD'] = pd.to_datetime(df_futures_corn['TIME_PERIOD'], format='%d/%m/%Y')
df_futures_corn['TIME_PERIOD'] = df_futures_corn['TIME_PERIOD'].dt.strftime('%Y/%d/%m')

df_futures_corn.drop(columns=['Open','High','Low','Vol.','Change %'], inplace=True)

df_futures_corn['TIME_PERIOD'] = pd.to_datetime(df_futures_corn['TIME_PERIOD'])

df_futures_corn.sort_values(by='TIME_PERIOD', ascending=True, inplace=True)
df_futures_corn

,TIME_PERIOD,Corn Price Futures
212,2005-02-01,222.75
211,2005-03-01,221.00
210,2005-04-01,213.50
209,2005-05-01,230.75
208,2005-06-01,222.25
...,...,...
4,2022-06-01,743.75
3,2022-07-01,616.25
2,2022-08-01,673.75
1,2022-09-01,677.50


## MACROECONOMIC DATA

In [7]:
#import datetime
import numpy
import openpyxl
#import data
path_file_1_p = "macroeconomic_data/Population_projections.xlsx"
population_data = pd.read_excel(path_file_1_p)

path_file_1 = "macroeconomic_data/macro_economic_indicators.csv"
economic_indicators = pd.read_csv(path_file_1, sep = ";", decimal=".")

#rename column for merge later
population_data = population_data.rename(columns = {'Country Name': 'PARTNER_Labels'})
economic_indicators = economic_indicators.rename(columns = {'country_name': 'PARTNER_Labels' ,'Time':'TIME_PERIOD' })
#this is a quick fix
macro_economic_indicators = economic_indicators
# Convert year to datetime with year and month
macro_economic_indicators['year'] = pd.to_datetime(macro_economic_indicators['year'], format='%Y').dt.to_period('M')
# Set the date as the index
macro_economic_indicators.set_index('year', inplace=True)

#make df for monthly data
monthly_data = macro_economic_indicators.iloc[0:0].copy()


number_of_col = macro_economic_indicators.shape[1]
nan_row = numpy.empty((number_of_col,))
nan_row[:] = numpy.nan

#add na rows for 11 months in a year
n_row = macro_economic_indicators.shape[0] 
for i in range(n_row):
    print(i)
    holder_for_rows = macro_economic_indicators.iloc[i,:]
    current_country = macro_economic_indicators.iloc[i,0] 
    current_country_code = macro_economic_indicators.iloc[i,1] 
    for j in range(12):
        index_3 = 12 * i + j
        if j == 0 :
            monthly_data.loc[index_3] = holder_for_rows   
        else:
            monthly_data.loc[index_3] = nan_row
            monthly_data.iloc[index_3,0] = current_country
            monthly_data.iloc[index_3,1] = current_country_code

#slects als the column with numbers
c = monthly_data.columns[3:12]
monthly_data[c] = monthly_data[c].apply(pd.to_numeric)

n_countries = len(population_data['Country Code'].unique())
#fills in the date column
for i in range(n_countries-1):
    index_1 = 0   + 204*i
    index_2 = 204 + 204*i
    monthly_data.iloc[index_1:index_2,:] = monthly_data.iloc[index_1:index_2,:].interpolate().copy()
    #adding dates 
    monthly_data.iloc[index_1:index_2,2] = pd.period_range("2005-01", "2021-12", freq='M')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701


In [8]:
monthly_data  = monthly_data.rename(columns = {'timecode': 'TIME_PERIOD'})
# convert Period and float values to string format, and then to datetime
monthly_data['TIME_PERIOD'] = monthly_data['TIME_PERIOD'].apply(lambda x: str(x) if isinstance(x, pd.Period) else x)
monthly_data['TIME_PERIOD'] = pd.to_datetime(monthly_data['TIME_PERIOD'], errors='coerce')

df_macroeconomic = monthly_data
df_macroeconomic

,PARTNER_Labels,country_code,TIME_PERIOD,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total
year,,,,,,,,,,,
0,Argentina,ARG,2005-01-01,1.987371e+11,4.352966e+11,39070501.0,11.510000,1.299209e+06,9.640000,1712.850000,89.050000
1,Argentina,ARG,2005-02-01,2.015554e+11,4.382157e+11,39104363.5,11.390833,1.299391e+06,9.601667,1723.801667,89.016667
2,Argentina,ARG,2005-03-01,2.043738e+11,4.411348e+11,39138226.0,11.271667,1.299572e+06,9.563333,1734.753333,88.983333
3,Argentina,ARG,2005-04-01,2.071921e+11,4.440538e+11,39172088.5,11.152500,1.299754e+06,9.525000,1745.705000,88.950000
4,Argentina,ARG,2005-05-01,2.100105e+11,4.469729e+11,39205951.0,11.033333,1.299936e+06,9.486667,1756.656667,88.916667
...,...,...,...,...,...,...,...,...,...,...,...
54319,Last Updated: 12/22/2022,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54320,Last Updated: 12/22/2022,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54321,Last Updated: 12/22/2022,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## MERGING DATA

### Merged trade data, oil price

In [36]:
df_maize['TIME_PERIOD'] = pd.to_datetime(df_maize['TIME_PERIOD'])
df_merged_trade_oil = df_oil_price.merge(df_maize, on='TIME_PERIOD', how='outer')
df_merged_trade_oil.head(30)

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price
0,2005-01-01,42.972278,Argentina,maize,8100.0,937.50,0.115741
1,2005-01-01,42.972278,Belgium,maize,6402.3,762.53,0.119103
2,2005-01-01,42.972278,France,maize,106983.9,13119.68,0.122632
3,2005-01-01,42.972278,Germany,maize,39590.1,4767.79,0.120429
4,2005-01-01,42.972278,Hungary,maize,4705.2,515.71,0.109604
5,2005-01-01,42.972278,India,maize,0.8,2.64,3.300000
6,2005-01-01,42.972278,Italy,maize,120.1,4.56,0.037968
7,2005-01-01,42.972278,Kenya,maize,1.8,4.65,2.583333
8,2005-01-01,42.972278,Peru,maize,1.6,6.36,3.975000
9,2005-01-01,42.972278,Tanzania,maize,0.4,1.60,4.000000


### Merged trade data, oil price, weather data

In [10]:
df_merged_trade_oil_weather = df_merged_trade_oil.merge(df_global_temperature, on=['TIME_PERIOD', 'PARTNER_Labels'], how='outer')
df_merged_trade_oil_weather

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,AVG_TAVG,MIN_TMIN,MAX_TMAX
0,2005-01-01,42.972278,Argentina,maize,8100.0,937.50,0.115741,NaN,NaN,NaN
1,2005-01-01,42.972278,Belgium,maize,6402.3,762.53,0.119103,30.700000,-31.000000,79.000000
2,2005-01-01,42.972278,France,maize,106983.9,13119.68,0.122632,50.081198,-40.377358,124.038462
3,2005-01-01,42.972278,Germany,maize,39590.1,4767.79,0.120429,28.651592,-91.000000,129.842105
4,2005-01-01,42.972278,Hungary,maize,4705.2,515.71,0.109604,-1.306452,-136.000000,137.000000
...,...,...,...,...,...,...,...,...,...,...
21476,2022-08-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,287.060676,244.363636,348.666667
21477,2022-09-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,279.751352,233.600000,350.000000
21478,2022-10-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,246.000000,238.000000,260.000000
21479,2022-11-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,272.000000,252.000000,304.000000


### Merged trade data, oil price, weather, future prices data

In [11]:
df_merged_trade_oil_weather_futures = df_merged_trade_oil_weather.merge(df_futures_corn, on='TIME_PERIOD', how='outer')
df_merged_trade_oil_weather_futures

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,AVG_TAVG,MIN_TMIN,MAX_TMAX,Corn Price Futures
0,2005-01-01,42.972278,Argentina,maize,8100.0,937.50,0.115741,NaN,NaN,NaN,NaN
1,2005-01-01,42.972278,Belgium,maize,6402.3,762.53,0.119103,30.700000,-31.000000,79.000000,NaN
2,2005-01-01,42.972278,France,maize,106983.9,13119.68,0.122632,50.081198,-40.377358,124.038462,NaN
3,2005-01-01,42.972278,Germany,maize,39590.1,4767.79,0.120429,28.651592,-91.000000,129.842105,NaN
4,2005-01-01,42.972278,Hungary,maize,4705.2,515.71,0.109604,-1.306452,-136.000000,137.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
21476,2022-12-01,NaN,Ukraine,NaN,NaN,NaN,NaN,33.276307,-78.000000,163.333333,NaN
21477,2022-12-01,NaN,United States,NaN,NaN,NaN,NaN,-14.536099,-125.184513,64.457218,NaN
21478,2022-12-01,NaN,Burkina Faso,NaN,NaN,NaN,NaN,260.295430,160.333333,373.500000,NaN
21479,2022-12-01,NaN,Uzbekistan,NaN,NaN,NaN,NaN,-32.452923,-173.700000,113.750000,NaN


### Merged trade, oil price, weather, future prices, macroeconomic 

In [12]:
df_merged = df_merged_trade_oil_weather_futures.merge(df_macroeconomic, on=['TIME_PERIOD', 'PARTNER_Labels'], how='left')
df_merged

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,AVG_TAVG,MIN_TMIN,MAX_TMAX,Corn Price Futures,country_code,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total
0,2005-01-01,42.972278,Argentina,maize,8100.0,937.50,0.115741,NaN,NaN,NaN,NaN,ARG,1.987371e+11,4.352966e+11,39070501.0,11.51,1299209.0,9.64,1712.85,89.05
1,2005-01-01,42.972278,Belgium,maize,6402.3,762.53,0.119103,30.700000,-31.000000,79.000000,NaN,BEL,3.857148e+11,4.026519e+11,10478617.0,8.44,13830.0,2.46,5555.62,74.76
2,2005-01-01,42.972278,France,maize,106983.9,13119.68,0.122632,50.081198,-40.377358,124.038462,NaN,FRA,2.196945e+12,2.224791e+12,63188395.0,8.88,293904.0,8.66,4286.54,52.30
3,2005-01-01,42.972278,Germany,maize,39590.1,4767.79,0.120429,28.651592,-91.000000,129.842105,NaN,DEU,2.846864e+12,2.912034e+12,82469422.0,11.17,170310.0,7.28,4086.50,81.86
4,2005-01-01,42.972278,Hungary,maize,4705.2,515.71,0.109604,-1.306452,-136.000000,137.000000,NaN,HUN,1.132112e+11,1.137681e+11,10087065.0,7.19,58630.0,7.29,2731.04,80.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21476,2022-12-01,NaN,Ukraine,NaN,NaN,NaN,NaN,33.276307,-78.000000,163.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21477,2022-12-01,NaN,United States,NaN,NaN,NaN,NaN,-14.536099,-125.184513,64.457218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21478,2022-12-01,NaN,Burkina Faso,NaN,NaN,NaN,NaN,260.295430,160.333333,373.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21479,2022-12-01,NaN,Uzbekistan,NaN,NaN,NaN,NaN,-32.452923,-173.700000,113.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## CLEANING EXTREME VALUES

In [13]:
import numpy as np

# Delete all inf and -inf
df_merged = df_merged.replace([np.inf, -np.inf], np.nan)

In [37]:
# Define a function to replace extreme high values with NaN
def replace_outliers(df, column):
    z = np.abs((df[column] - df[column].mean()) / df[column].std()) # Calculate z-score for the column
    outliers = df[z > 3][column] # Identify extreme high values with a z-score greater than 3
    median = df[column].median() # Calculate the median of the column
    df.loc[z > 3, column] = np.nan # Replace extreme high values with NaN

# Define a list of columns to perform outlier treatment on
columns = ['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures', 'gdp_current_us', 'population_total', 'unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total', 'energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total']

# Iterate over the columns and call the replace_outliers function for each one
for column in columns:
    replace_outliers(df_merged, column)
    

## FULL DATAFRAME (without filtering top countries)

In [15]:
###FOR NOW
df_maize_total = df_merged.pivot_table(index='TIME_PERIOD', columns=['PARTNER_Labels'], values=['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total'], margins=True)
df_maize_total

AVG_TAVG                                             \
PARTNER_Labels       Albania     Algeria Angola     Armenia   Australia   
TIME_PERIOD                                                               
2005-01-01 00:00:00      NaN   76.228202  274.0  -87.478261  242.290980   
2005-02-01 00:00:00      NaN   84.478184  270.5  -87.000000  237.755531   
2005-03-01 00:00:00      NaN  147.382230  156.0  -46.440000  223.480383   
2005-04-01 00:00:00      NaN  181.541272  242.5   39.714286  208.498275   
2005-05-01 00:00:00      NaN  240.048870  211.5   96.043478  167.861508   
...                      ...         ...    ...         ...         ...   
2022-09-01 00:00:00      NaN  286.008954    NaN  173.108906  165.721311   
2022-10-01 00:00:00      NaN  231.950455    NaN  111.054056  189.637400   
2022-11-01 00:00:00      NaN  168.537960    NaN   50.784508  201.174639   
2022-12-01 00:00:00      NaN  150.914659    NaN   -1.926037  224.700188   
All                      NaN         NaN    NaN         NaN         NaN   

                                                                     \
PARTNER_Labels          Austria  Azerbaijan     Bahrain     Belarus   
TIME_PERIOD                                                           
2005-01-01 00:00:00   -1.166667         NaN  160.222222   -7.788025   
2005-02-01 00:00:00  -42.166667   72.000000  176.000000  -59.820327   
2005-03-01 00:00:00   -1.994624   68.000000  211.095238  -32.497550   
2005-04-01 00:00:00   60.233333  191.000000  257.315789   80.398109   
2005-05-01 00:00:00  108.758065         NaN  303.117647  130.261818   
...                         ...         ...         ...         ...   
2022-09-01 00:00:00  103.779082  218.628333  346.350000   99.815254   
2022-10-01 00:00:00   98.433871  160.495556  307.000000   82.525416   
2022-11-01 00:00:00   36.939583   96.659524  274.357143   17.335752   
2022-12-01 00:00:00   -9.296552   37.345238  223.176471  -29.105331   
All                   78.260970         NaN         NaN         NaN   

                                 ... unemployment_total            \
PARTNER_Labels          Belgium  ...             Uganda   Ukraine   
TIME_PERIOD                      ...                                
2005-01-01 00:00:00   30.700000  ...                NaN  7.180000   
2005-02-01 00:00:00   20.000000  ...                NaN  7.149167   
2005-03-01 00:00:00   55.800000  ...                NaN  7.118333   
2005-04-01 00:00:00  111.000000  ...                NaN  7.087500   
2005-05-01 00:00:00  129.250000  ...                NaN  7.056667   
...                         ...  ...                ...       ...   
2022-09-01 00:00:00  166.000000  ...                NaN       NaN   
2022-10-01 00:00:00  147.333333  ...                NaN       NaN   
2022-11-01 00:00:00   88.454545  ...                NaN       NaN   
2022-12-01 00:00:00   36.900000  ...                NaN       NaN   
All                  111.602196  ...                NaN  8.515025   

                                                                               \
PARTNER_Labels      United Arab Emirates United Kingdom United States Uruguay   
TIME_PERIOD                                                                     
2005-01-01 00:00:00             3.120000       4.750000      5.080000     NaN   
2005-02-01 00:00:00             3.108788       4.800000      5.041667     NaN   
2005-03-01 00:00:00             3.097576       4.850000      5.003333     NaN   
2005-04-01 00:00:00             3.086364       4.900000      4.965000     NaN   
2005-05-01 00:00:00             3.075152       4.950000      4.926667     NaN   
...                                  ...            ...           ...     ...   
2022-09-01 00:00:00                  NaN            NaN           NaN     NaN   
2022-10-01 00:00:00                  NaN            NaN           NaN     NaN   
2022-11-01 00:00:00                  NaN            NaN           NaN     NaN   
2022-12-01 00:00:00                

In [16]:
# Create a new DataFrame that only contains the 'price' values of France and Germany:
df_price = df_maize_total[['price']].loc[:, ('price', ['France', 'Germany'])]

In [17]:
# Remove any rows that contain missing values (NaN) in the 'price' column for France and Germany
df_price = df_price.dropna()

In [18]:
df_price_mean = df_price.mean(axis=1)
df_price_mean

TIME_PERIOD
2005-01-01 00:00:00    0.121531
2005-02-01 00:00:00    0.116532
2005-03-01 00:00:00    0.121236
2005-04-01 00:00:00    0.113698
2005-05-01 00:00:00    0.120077
                         ...   
2022-07-01 00:00:00    0.372453
2022-08-01 00:00:00    0.315314
2022-09-01 00:00:00    0.295308
2022-10-01 00:00:00    0.383753
All                    0.194901
Length: 215, dtype: float64

In [19]:
# Calculate the year-over-year percent change in 'price' for France and Germany:
df_price_pct_change = df_price.pct_change()

# Replace NaN values with 0 (there are NaN values for the first year)
df_price_pct_change = df_price_pct_change.fillna(0)
df_price_pct_change

# Calculate the mean percent change in 'price' for each year across France and Germany:
df_price_pct_change_mean = df_price_pct_change.mean(axis=1)



In [20]:
# Create a new DataFrame that only contains the missing 'price' values for other countries:
df_missing_price = df_maize_total[['price']].loc[:, ('price', df_maize_total['price'].isna().any())]
df_missing_price

price                                \
PARTNER_Labels      Antigua and Barbuda Argentina Australia   Austria   
TIME_PERIOD                                                             
2005-01-01 00:00:00                 NaN  0.115741       NaN       NaN   
2005-02-01 00:00:00                 NaN  0.123755       NaN       NaN   
2005-03-01 00:00:00                 NaN  0.120312       NaN  0.133137   
2005-04-01 00:00:00                 NaN  0.109644       NaN       NaN   
2005-05-01 00:00:00                 NaN  0.130592       NaN       NaN   
...                                 ...       ...       ...       ...   
2022-09-01 00:00:00                 NaN  0.441667       NaN  0.539147   
2022-10-01 00:00:00                 NaN  0.444080       NaN  0.554460   
2022-11-01 00:00:00                 NaN       NaN       NaN       NaN   
2022-12-01 00:00:00                 NaN       NaN       NaN       NaN   
All                                 NaN       NaN       NaN  1.752232   

                                                                           \
PARTNER_Labels      Bangladesh   Belgium Bosnia and Herzegovina    Brazil   
TIME_PERIOD                                                                 
2005-01-01 00:00:00        NaN  0.119103                    NaN       NaN   
2005-02-01 00:00:00        NaN  0.118607                    NaN  0.164250   
2005-03-01 00:00:00        NaN  0.121043                    NaN       NaN   
2005-04-01 00:00:00        NaN  0.122473                    NaN       NaN   
2005-05-01 00:00:00        NaN  0.129335                    NaN       NaN   
...                        ...       ...                    ...       ...   
2022-09-01 00:00:00        NaN  0.421045                    NaN  0.296534   
2022-10-01 00:00:00        NaN  0.399394                    NaN  0.314287   
2022-11-01 00:00:00        NaN       NaN                    NaN       NaN   
2022-12-01 00:00:00        NaN       NaN                    NaN       NaN   
All                        NaN  0.218329                    0.4  0.373898   

                                       ...                   \
PARTNER_Labels       Bulgaria Burundi  ... Uganda   Ukraine   
TIME_PERIOD                            ...                    
2005-01-01 00:00:00       NaN     NaN  ...    NaN       NaN   
2005-02-01 00:00:00       NaN     NaN  ...    NaN       NaN   
2005-03-01 00:00:00       NaN     NaN  ...    NaN       NaN   
2005-04-01 00:00:00       NaN     NaN  ...    NaN       NaN   
2005-05-01 00:00:00       NaN     NaN  ...    NaN       NaN   
...                       ...     ...  ...    ...       ...   
2022-09-01 00:00:00  0.540323     NaN  ...    NaN  0.313625   
2022-10-01 00:00:00  0.455789     NaN  ...    NaN  0.313023   
2022-11-01 00:00:00       NaN     NaN  ...    NaN       NaN   
2022-12-01 00:00:00       NaN     NaN  ...    NaN       NaN   
All                  1.849431     NaN  ...    NaN  0.203271   

                                                                               \
PARTNER_Labels      United Arab Emirates United Kingdom United States Uruguay   
TIME_PERIOD                                                                     
2005-01-01 00:00:00                  NaN       0.314011      0.334963     NaN   
2005-02-01 00:00:00                  NaN       0.380808      0.270379     NaN   
2005-03-01 00:00:00                  NaN       0.329100      0.274034     NaN   
2005-04-01 00:00:00                  NaN       0.292759      0.365034     NaN   
2005-05-01 00:00:00                  NaN       0.336029      0.298552     NaN   
...                                  ...            ...           ...     ...   
2022-09-01 00:00:00                  NaN       0.480000      0.430013     NaN   
2022-10-01 00:00:00                  NaN       0.449057      0.443971     NaN   
2022-11-01 00:00:00                  NaN            NaN           NaN     NaN   
2022-12-01 00:00:00                  NaN            NaN           NaN     NaN

In [21]:
# Fill in the missing 'price' values for each year in the new DataFrame using the mean percent change from France and Germany
df_missing_price = df_missing_price.fillna(method='ffill')
df_missing_price = df_missing_price.fillna(method='bfill')
df_missing_price = df_missing_price.fillna(df_price.iloc[-1])
df_missing_price = df_missing_price.apply(lambda x: x * (1 + df_price_pct_change_mean))

df_missing_price

price                                \
PARTNER_Labels      Antigua and Barbuda Argentina Australia   Austria   
TIME_PERIOD                                                             
2005-01-01 00:00:00            0.392593  0.115741  0.157143  0.133137   
2005-02-01 00:00:00            0.376255  0.118605  0.150604  0.127597   
2005-03-01 00:00:00            0.409603  0.125525  0.163951  0.138906   
2005-04-01 00:00:00            0.368089  0.102801  0.147335  0.124828   
2005-05-01 00:00:00            0.414697  0.137945  0.165991  0.140634   
...                                 ...       ...       ...       ...   
2022-09-01 00:00:00            0.440136  0.432341  0.587332  0.527764   
2022-10-01 00:00:00            0.699595  0.690960  0.933562  0.862705   
2022-11-01 00:00:00                 NaN       NaN       NaN       NaN   
2022-12-01 00:00:00                 NaN       NaN       NaN       NaN   
All                            0.233877  0.230991  0.312093  0.911433   

                                                                           \
PARTNER_Labels      Bangladesh   Belgium Bosnia and Herzegovina    Brazil   
TIME_PERIOD                                                                 
2005-01-01 00:00:00   0.175000  0.119103               0.400000  0.164250   
2005-02-01 00:00:00   0.167718  0.113671               0.383355  0.157415   
2005-03-01 00:00:00   0.182582  0.126287               0.417331  0.171367   
2005-04-01 00:00:00   0.164077  0.114829               0.375034  0.153998   
2005-05-01 00:00:00   0.184853  0.136617               0.422521  0.173498   
...                        ...       ...                    ...       ...   
2022-09-01 00:00:00   0.171305  0.412155               0.391554  0.290273   
2022-10-01 00:00:00   0.272289  0.621432               0.622375  0.489011   
2022-11-01 00:00:00        NaN       NaN                    NaN       NaN   
2022-12-01 00:00:00        NaN       NaN                    NaN       NaN   
All                   0.091027  0.113565               0.208062  0.194485   

                                         ...                      \
PARTNER_Labels       Bulgaria   Burundi  ...    Uganda   Ukraine   
TIME_PERIOD                              ...                       
2005-01-01 00:00:00  0.224834  0.209091  ...  1.200000  0.203184   
2005-02-01 00:00:00  0.215478  0.200390  ...  1.150064  0.194729   
2005-03-01 00:00:00  0.234575  0.218150  ...  1.251993  0.211988   
2005-04-01 00:00:00  0.210801  0.196040  ...  1.125102  0.190502   
2005-05-01 00:00:00  0.237493  0.220864  ...  1.267564  0.214624   
...                       ...       ...  ...       ...       ...   
2022-09-01 00:00:00  0.528914  0.204676  ...  0.538387  0.307003   
2022-10-01 00:00:00  0.709180  0.325332  ...  0.855765  0.487044   
2022-11-01 00:00:00       NaN       NaN  ...       NaN       NaN   
2022-12-01 00:00:00       NaN       NaN  ...       NaN       NaN   
All                  0.961992  0.108760  ...  0.286086  0.105733   

                                                                       \
PARTNER_Labels      United Arab Emirates United Kingdom United States   
TIME_PERIOD                                                             
2005-01-01 00:00:00             0.242762       0.314011      0.334963   
2005-02-01 00:00:00             0.232660       0.364961      0.259128   
2005-03-01 00:00:00             0.253280       0.343359      0.285907   
2005-04-01 00:00:00             0.227610       0.274487      0.342251   
2005-05-01 00:00:00             0.256430       0.354949      0.315362   
...                                  ...            ...           ...   
2022-09-01 00:00:00             0.440499       0.469865      0.420934   
2022-10-01 00:00:00             0.700172       0.698704      0.690790   
2022-11-01 00:00:00                  NaN            NaN           NaN   
2022-12-01 00:00:00                  NaN            NaN           NaN   
All                           

In [22]:
# Update the original DataFrame with the filled-in missing values

df_maize_total.update(df_missing_price)
df_maize_total

AVG_TAVG                                             \
PARTNER_Labels       Albania     Algeria Angola     Armenia   Australia   
TIME_PERIOD                                                               
2005-01-01 00:00:00      NaN   76.228202  274.0  -87.478261  242.290980   
2005-02-01 00:00:00      NaN   84.478184  270.5  -87.000000  237.755531   
2005-03-01 00:00:00      NaN  147.382230  156.0  -46.440000  223.480383   
2005-04-01 00:00:00      NaN  181.541272  242.5   39.714286  208.498275   
2005-05-01 00:00:00      NaN  240.048870  211.5   96.043478  167.861508   
...                      ...         ...    ...         ...         ...   
2022-09-01 00:00:00      NaN  286.008954    NaN  173.108906  165.721311   
2022-10-01 00:00:00      NaN  231.950455    NaN  111.054056  189.637400   
2022-11-01 00:00:00      NaN  168.537960    NaN   50.784508  201.174639   
2022-12-01 00:00:00      NaN  150.914659    NaN   -1.926037  224.700188   
All                      NaN         NaN    NaN         NaN         NaN   

                                                                     \
PARTNER_Labels          Austria  Azerbaijan     Bahrain     Belarus   
TIME_PERIOD                                                           
2005-01-01 00:00:00   -1.166667         NaN  160.222222   -7.788025   
2005-02-01 00:00:00  -42.166667   72.000000  176.000000  -59.820327   
2005-03-01 00:00:00   -1.994624   68.000000  211.095238  -32.497550   
2005-04-01 00:00:00   60.233333  191.000000  257.315789   80.398109   
2005-05-01 00:00:00  108.758065         NaN  303.117647  130.261818   
...                         ...         ...         ...         ...   
2022-09-01 00:00:00  103.779082  218.628333  346.350000   99.815254   
2022-10-01 00:00:00   98.433871  160.495556  307.000000   82.525416   
2022-11-01 00:00:00   36.939583   96.659524  274.357143   17.335752   
2022-12-01 00:00:00   -9.296552   37.345238  223.176471  -29.105331   
All                   78.260970         NaN         NaN         NaN   

                                 ... unemployment_total            \
PARTNER_Labels          Belgium  ...             Uganda   Ukraine   
TIME_PERIOD                      ...                                
2005-01-01 00:00:00   30.700000  ...                NaN  7.180000   
2005-02-01 00:00:00   20.000000  ...                NaN  7.149167   
2005-03-01 00:00:00   55.800000  ...                NaN  7.118333   
2005-04-01 00:00:00  111.000000  ...                NaN  7.087500   
2005-05-01 00:00:00  129.250000  ...                NaN  7.056667   
...                         ...  ...                ...       ...   
2022-09-01 00:00:00  166.000000  ...                NaN       NaN   
2022-10-01 00:00:00  147.333333  ...                NaN       NaN   
2022-11-01 00:00:00   88.454545  ...                NaN       NaN   
2022-12-01 00:00:00   36.900000  ...                NaN       NaN   
All                  111.602196  ...                NaN  8.515025   

                                                                               \
PARTNER_Labels      United Arab Emirates United Kingdom United States Uruguay   
TIME_PERIOD                                                                     
2005-01-01 00:00:00             3.120000       4.750000      5.080000     NaN   
2005-02-01 00:00:00             3.108788       4.800000      5.041667     NaN   
2005-03-01 00:00:00             3.097576       4.850000      5.003333     NaN   
2005-04-01 00:00:00             3.086364       4.900000      4.965000     NaN   
2005-05-01 00:00:00             3.075152       4.950000      4.926667     NaN   
...                                  ...            ...           ...     ...   
2022-09-01 00:00:00                  NaN            NaN           NaN     NaN   
2022-10-01 00:00:00                  NaN            NaN           NaN     NaN   
2022-11-01 00:00:00                  NaN            NaN           NaN     NaN   
2022-12-01 00:00:00                

## HANDLE MISSING DATA

### price

## Save excel file

In [23]:
# Set the path and name of the Excel file
file_path = "df_maize_total.csv"

# Write the df_pivot dataframe to the specified Excel file
df_maize_total.to_csv(file_path, index=True)

In [24]:
df_maize_total.to_excel("maize_missingpricesfilled.xlsx")

## ANALYSIS CORN IMPORT TO NL (HISTORICAL) & FILTER TOP PARTNERS

In [25]:
df_maize_corn_0['TIME_PERIOD'] = pd.to_datetime(df_maize_corn_0['TIME_PERIOD'])
mask = (df_maize_corn_0['TIME_PERIOD'] >= '2005-01-01') & (df_maize_corn_0['TIME_PERIOD'] <= '2022-10-01')
df_filtered = df_maize_corn_0.loc[mask]
total = df_filtered['OBS_VALUE_QUANTITY_TON'].sum()

total

77296299.4

In [26]:
df_maize_corn_0['TIME_PERIOD'] = pd.to_datetime(df_maize_corn_0['TIME_PERIOD'])
mask = (df_maize_corn_0['TIME_PERIOD'] >= '2005-01-01') & (df_maize_corn_0['TIME_PERIOD'] <= '2022-10-01')
df_filtered = df_maize_corn_0.loc[mask]

grouped = df_filtered.groupby('PARTNER_Labels')['OBS_VALUE_QUANTITY_TON'].sum().reset_index()
grouped = grouped.sort_values(by='OBS_VALUE_QUANTITY_TON', ascending=False)
grouped = grouped.reset_index()
grouped['cumulative_sum'] = grouped['OBS_VALUE_QUANTITY_TON'].cumsum()
grouped['cumulative_sum_pct'] = grouped['cumulative_sum'] / grouped['OBS_VALUE_QUANTITY_TON'].sum()

grouped.head(10)        
        


,index,PARTNER_Labels,OBS_VALUE_QUANTITY_TON,cumulative_sum,cumulative_sum_pct
0,101,Ukraine,22670363.9,22670363.9,0.293292
1,32,France,20818695.8,43489059.7,0.562628
2,9,Brazil,7514296.3,51003356.0,0.659842
3,35,Germany,7132645.5,58136001.5,0.752119
4,40,Hungary,3995159.3,62131160.8,0.803805
5,79,Romania,3816059.5,65947220.3,0.853174
6,6,Belgium,3008019.1,68955239.4,0.892090
7,80,Russia,1464272.2,70419511.6,0.911033
8,2,Argentina,1357260.3,71776771.9,0.928593
9,104,United States,1045888.5,72822660.4,0.942124


In [27]:
# Declare splitting point for cumulative import to NL history, biggest partners
splitting_point = 0.8

for i, row in grouped.iterrows():
    if row['cumulative_sum_pct'] >= splitting_point:
        top_x = i + 1
        break
        
top_partners = grouped['PARTNER_Labels'].head(top_x).tolist()
top_partners

['Ukraine', 'France', 'Brazil', 'Germany', 'Hungary']

## Simplicity/overview: filtered top partners

In [28]:
pd.set_option('display.max_columns', 90) #replace n with the number of columns you want to see completely

In [38]:
# filter the pivot table using the loc method
df_maize_total_filtered = df_maize_total.loc[:, (slice(None), top_partners)]


# GEEEN IDEE WAAROM, MAAR NA FILTEREN VERANDERT PIVOT OPEENS WEER NAAR VOLGORDE DIE WE EERST VOOR OGEN HADDEN, MWEH
# display the filtered pivot table
df_maize_total_filtered = df_maize_total_filtered.sort_index(axis=1)
df_maize_total_filtered.head(5)

AVG_TAVG                                      \
PARTNER_Labels           Brazil      France     Germany     Hungary   
TIME_PERIOD                                                           
2005-01-01 00:00:00  269.701032   50.081198   28.651592   -1.306452   
2005-02-01 00:00:00  266.743474   30.689158   -1.962406  -25.371429   
2005-03-01 00:00:00  266.528149   76.202630   45.730051   38.383871   
2005-04-01 00:00:00  253.719444  108.081105  101.859649  112.460000   
2005-05-01 00:00:00  247.859946  147.068032  134.667233  162.993548   

                                Corn Price Futures                          \
PARTNER_Labels          Ukraine             Brazil  France Germany Hungary   
TIME_PERIOD                                                                  
2005-01-01 00:00:00    0.869290                NaN     NaN     NaN     NaN   
2005-02-01 00:00:00  -38.376438             222.75  222.75  222.75  222.75   
2005-03-01 00:00:00   -3.794989             221.00  221.00  221.00  221.00   
2005-04-01 00:00:00   90.622214             213.50  213.50  213.50  213.50   
2005-05-01 00:00:00  161.493475             230.75  230.75  230.75  230.75   

                            MAX_TMAX                                      \
PARTNER_Labels      Ukraine   Brazil      France     Germany     Hungary   
TIME_PERIOD                                                                
2005-01-01 00:00:00     NaN   355.50  124.038462  129.842105  137.000000   
2005-02-01 00:00:00  222.75   342.50  119.983051  108.315789   57.666667   
2005-03-01 00:00:00  221.00   347.25  201.140625  180.894737  209.000000   
2005-04-01 00:00:00  213.50   338.50  254.950820  213.315789  218.000000   
2005-05-01 00:00:00  230.75   341.00  279.101695  314.684211  321.000000   

                                MIN_TMIN                                     \
PARTNER_Labels          Ukraine   Brazil     France     Germany     Hungary   
TIME_PERIOD                                                                   
2005-01-01 00:00:00  102.142857   182.00 -40.377358  -91.000000 -136.000000   
2005-02-01 00:00:00   60.142857   180.00 -65.083333 -125.684211 -167.600000   
2005-03-01 00:00:00  112.468208   201.75 -90.450000  -99.526316 -126.400000   
2005-04-01 00:00:00  204.189944   150.00  23.620690  -12.421053   -2.000000   
2005-05-01 00:00:00  307.400000   138.00  56.158730   22.631579   27.833333   

                                OBS_VALUE_QUANTITY_TON                     \
PARTNER_Labels          Ukraine                 Brazil    France  Germany   
TIME_PERIOD                                                                 
2005-01-01 00:00:00 -105.930233                    NaN  106983.9  39590.1   
2005-02-01 00:00:00 -190.275862                   40.0  126648.5  73452.7   
2005-03-01 00:00:00 -123.523810                    NaN   94526.9  66027.6   
2005-04-01 00:00:00  -37.809524                    NaN  137965.3  52878.4   
2005-05-01 00:00:00   57.571429                    NaN  122365.8  42554.1   

                                     OBS_VALUE_VALUE_1000EURO            \
PARTNER_Labels       Hungary Ukraine                   Brazil    France   
TIME_PERIOD                                                               
2005-01-01 00:00:00   4705.2     NaN                      NaN  13119.68   
2005-02-01 00:00:00  16053.2     NaN                     6.57  15708.00   
2005-03-01 00:00:00  14437.1     NaN                      NaN  11692.80   
2005-04-01 00:00:00   6912.4     0.0                      NaN  16202.52   
2005-05-01 00:00:00  19030.3     NaN                      NaN  15090.74   

                                              OIL_PRICE $/bbl             \
PARTNER_Labels       Germany  Hungary Ukraine          Brazil     France   
TIME_PERIOD                                                                
2005-01-01 00:00:00  4767.79   515.71     NaN             NaN  42.972278   
2005-02-01 00:00:00  8008.91  1694.86     NaN       44.818211  44

In [30]:
# Filter the data to include only the desired countries
#df_filtered = df_merged[df_merged['PARTNER_Labels'].isin(top_partners)]

# Create the pivot table
#df_filtered_maize_total = df_filtered.pivot_table(index='TIME_PERIOD',columns=['PARTNER_Labels'], values=['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total'])
#df_filtered_maize_total

In [42]:
df_maize_total_filtered.to_csv("cadeautjevoorLiza.csv", index=True)

In [32]:
df_maize_total_filtered.to_csv("cadeautjevoorGonemenLiza")

In [33]:
df_maize_total_filtered['price'].median()

PARTNER_Labels
Ukraine    0.197036
France     0.192336
Brazil     0.185076
Germany    0.204806
Hungary    0.177812
dtype: float64